In [13]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

In [3]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i,c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i,c in enumerate(self.chars))
        
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        
        return ''.join(self.indices_char[x] for x in x)
    
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [8]:
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

MAXLEN = DIGITS + 1 + DIGITS

chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("generating data...")

while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                           for i in range(np.random.randint(1, DIGITS + 1))))
    
    a, b = f(), f()
    
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    
    seen.add(key)
    
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a+b)
    
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    
    questions.append(query)
    expected.append(ans)
    
print('Total addition questions: ', len(questions))

generating data...
Total addition questions:  50000


In [19]:
print(questions[0])
print(expected[0])
print(questions[1])
print(expected[1])
print(questions[2])
print(expected[2])

  266+6
668 
 396+01
703 
  3+257
755 


In [9]:
print('Vectorization...')

x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)

for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
    
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [11]:
print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [14]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [15]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [21]:
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1, validation_data=(x_val, y_val))
    
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q',q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + ' O ' + colors.close, end=' ')
        else:
            print(colors.fail + ' X '+colors.close, end = ' ')
        
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 231us/step - loss: 1.5798 - acc: 0.4102 - val_loss: 1.4979 - val_acc: 0.4384
Q 186+521 T 707   X  818 
Q 636+833 T 1469  X  1328
Q 490+239 T 729   X  714 
Q 5+19    T 24    X  11  
Q 238+988 T 1226  X  1218
Q 98+86   T 184   X  188 
Q 161+33  T 194   X  274 
Q 91+522  T 613   X  501 
Q 463+36  T 499   X  441 
Q 23+223  T 246   X  334 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 230us/step - loss: 1.4227 - acc: 0.4677 - val_loss: 1.3408 - val_acc: 0.5021
Q 75+140  T 215   X  184 
Q 117+13  T 130   X  124 
Q 389+960 T 1349  X  1384
Q 938+55  T 993   X  900 
Q 653+638 T 1291  X  1366
Q 542+918 T 1460  X  1466
Q 575+57  T 632   X  544 
Q 599+13  T 612   X  566 
Q 922+932 T 1854  X  1780
Q 57

45000/45000 [==============================] - 12s 261us/step - loss: 0.1151 - acc: 0.9784 - val_loss: 0.1034 - val_acc: 0.9798
Q 15+945  T 960   X  961 
Q 3+329   T 332   O  332 
Q 80+41   T 121   O  121 
Q 263+593 T 856   O  856 
Q 54+107  T 161   O  161 
Q 129+364 T 493   O  493 
Q 64+49   T 113   O  113 
Q 22+45   T 67    X  77  
Q 406+715 T 1121  O  1121
Q 714+60  T 774   O  774 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 239us/step - loss: 0.0986 - acc: 0.9800 - val_loss: 0.0816 - val_acc: 0.9835
Q 70+59   T 129   O  129 
Q 83+71   T 154   O  154 
Q 740+12  T 752   O  752 
Q 574+366 T 940   X  930 
Q 8+3     T 11    X  12  
Q 81+735  T 816   O  816 
Q 9+366   T 375   O  375 
Q 763+9   T 772   O  772 
Q 76+894  T 970   O  970 
Q 783+55  T 838   O  838 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on

45000/45000 [==============================] - 10s 232us/step - loss: 0.0240 - acc: 0.9939 - val_loss: 0.0909 - val_acc: 0.9691
Q 758+12  T 770   O  770 
Q 716+7   T 723   O  723 
Q 48+71   T 119   O  119 
Q 923+7   T 930   O  930 
Q 13+436  T 449   X  459 
Q 49+627  T 676   O  676 
Q 780+51  T 831   O  831 
Q 860+20  T 880   O  880 
Q 704+722 T 1426  O  1426
Q 6+953   T 959   O  959 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 254us/step - loss: 0.0272 - acc: 0.9931 - val_loss: 0.0131 - val_acc: 0.9979
Q 657+482 T 1139  O  1139
Q 368+6   T 374   O  374 
Q 902+95  T 997   O  997 
Q 9+885   T 894   O  894 
Q 43+919  T 962   O  962 
Q 604+566 T 1170  O  1170
Q 984+63  T 1047  O  1047
Q 37+141  T 178   O  178 
Q 276+463 T 739   O  739 
Q 85+70   T 155   O  155 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on

45000/45000 [==============================] - 11s 242us/step - loss: 0.0025 - acc: 0.9999 - val_loss: 0.0047 - val_acc: 0.9991
Q 962+731 T 1693  O  1693
Q 468+859 T 1327  O  1327
Q 75+24   T 99    O  99  
Q 1+706   T 707   O  707 
Q 90+582  T 672   O  672 
Q 8+168   T 176   O  176 
Q 22+556  T 578   O  578 
Q 962+41  T 1003  O  1003
Q 465+7   T 472   O  472 
Q 495+773 T 1268  O  1268

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 232us/step - loss: 0.0023 - acc: 0.9999 - val_loss: 0.0050 - val_acc: 0.9989
Q 433+4   T 437   O  437 
Q 503+9   T 512   O  512 
Q 202+72  T 274   O  274 
Q 182+9   T 191   O  191 
Q 955+1   T 956   O  956 
Q 371+9   T 380   O  380 
Q 97+156  T 253   O  253 
Q 45+99   T 144   O  144 
Q 364+25  T 389   O  389 
Q 21+251  T 272   O  272 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on

45000/45000 [==============================] - 10s 231us/step - loss: 0.0402 - acc: 0.9882 - val_loss: 0.0518 - val_acc: 0.9819
Q 204+280 T 484   O  484 
Q 124+45  T 169   O  169 
Q 554+176 T 730   O  730 
Q 464+843 T 1307  O  1307
Q 39+622  T 661   O  661 
Q 57+663  T 720   O  720 
Q 345+893 T 1238  X  1138
Q 465+42  T 507   O  507 
Q 698+22  T 720   O  720 
Q 692+925 T 1617  O  1617

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 233us/step - loss: 0.0085 - acc: 0.9980 - val_loss: 0.0044 - val_acc: 0.99930.0
Q 753+61  T 814   O  814 
Q 54+109  T 163   O  163 
Q 446+893 T 1339  O  1339
Q 26+422  T 448   O  448 
Q 27+690  T 717   O  717 
Q 496+9   T 505   O  505 
Q 307+38  T 345   O  345 
Q 67+803  T 870   O  870 
Q 134+1   T 135   O  135 
Q 738+8   T 746   O  746 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate

KeyboardInterrupt: 